# Initialization

In [ ]:
from pathlib import Path
from datetime import datetime
import helper
import yaml, pandas as pd, numpy as np, tqdm.auto as tqdm, shutil, subprocess
from helper import RenderJSON
import config_adapter


In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
# Autofilled by calling script
param_path = "_" # Path to param config file
scripts_folder = "_" # Path to folder containing all scripts

In [ ]:
param_path = Path(param_path)
scripts_folder = Path(scripts_folder)
params = config_adapter.load(param_path)
RenderJSON(params)

In [ ]:
if "variables" in params:
    variables = config_adapter.normalize_yaml_paramlist(params["variables"], format=config_adapter.variable_param_format)
else:
    variables = {}
# handled_params = helper.replace_vals(params, variables)
ctx = config_adapter.Context()
ctx.methods["raw"] = config_adapter.raw
ctx.methods["expand_envvars"] = config_adapter.expand_envvars
for var in variables:
    config_adapter.add_variable_context(ctx, var)
RenderJSON(ctx.variables)


In [ ]:
if "tables" in params:
    tables = config_adapter.normalize_yaml_paramlist(params["tables"], format=config_adapter.table_param_format)
else: 
    tables = []
ctx.methods["find_files"] = config_adapter.find_files
ctx.methods["regex_filter"] = config_adapter.regex_filter
ctx.methods["from_rows"] = config_adapter.from_rows
ctx.methods["longest_prefix_join"] = config_adapter.longest_prefix_join

for table in tables:
    it_table = config_adapter.handle_duplicate_over(config_adapter.get_duplicate_table(ctx, table), table)
    for t in it_table:
        config_adapter.add_table_context(ctx, t)
for n, t in ctx.tables.items():
    t.style.set_caption(n)
    display(t)

In [ ]:
runs = [ctx.evaluate(r) for i in params["runs"] for r in config_adapter.handle_duplicate_over(config_adapter.get_duplicate_table(ctx, i), i) ]
display(RenderJSON(runs))
run_df = pd.DataFrame(runs).assign(run_num=np.arange(len(runs)))
run_df["script"] =  run_df["script"].apply(lambda s: helper.singleglob(scripts_folder, s).resolve())
run_df["script_name"] =  run_df["script"].apply(lambda s: s.name)
run_df["already_done"] = run_df["run_folder"].apply(lambda p: Path(p).exists() and not Path(p+".tmp").exists())
run_df 

In [ ]:
# run_df = pd.DataFrame(handled_params["runs"]).assign(run_num=np.arange(len(handled_params["runs"])))
# run_df.insert(0, "run_num", run_df.pop("run_num"))
# if run_df["run_folder"].duplicated().any():
#     raise Exception("run folder must be different")
# run_df["imports"] = run_df.pop("imports")
# run_df["script_params"] = run_df.pop("script_params")
# run_df
    

In [ ]:
infos = []
for _, row in tqdm.tqdm(run_df.to_dict(orient="index").items()):
    current_info = {}
    if "id" in row:
        current_info["id"] = row["id"]
    current_info["run_num"] = row["run_num"]
    script = row["script"]
    current_info["script_name"] = row["script_name"]
    if "requires_sucess" in row and (isinstance(row["requires_sucess"], list) or not pd.isna(row["requires_sucess"]) ):
        if not isinstance(row["requires_sucess"], list):
            row["requires_sucess"] = [row["requires_sucess"]]
        should_run=True
        for id in row["requires_sucess"]:
            i = [info["sucess"] for info in infos if "id" in info and info["id"]==id]
            if len(i) !=1:
                current_info["sucess"] = f"requires_sucess_find_problem(found {len(i)})"
                should_run=False
            elif not (i[0] == True or i[0] == "skipped"):
                current_info["sucess"] = "requirements_not_satisfied"
                should_run=False
        if not should_run:
            infos.append(current_info)
            continue
    run_folder = Path(row["run_folder"]+".tmp")
    dest_folder = Path(row["run_folder"])
    if (not row["already_done"]) or ("force_recompute" in row.keys() and row["force_recompute"]):
        try:
            if run_folder.exists():
                shutil.rmtree(run_folder)
            
            run_folder.mkdir(exist_ok=True, parents=True)
            if "imports" in row:
                if not isinstance(row["imports"], list):
                    row["imports"] = [row["imports"]]
                for im in row["imports"]:
                    fl = scripts_folder.glob(im)
                    for f in fl:
                        shutil.copyfile(f, run_folder / f.name)
            with (run_folder / "params.yaml").open("w") as f:
                yaml.dump(row["script_params"], f)
            env = row['environment']  if "environment" in row else subprocess.run('echo $CONDA_DEFAULT_ENV', shell=True, stdout=subprocess.PIPE).stdout
            current_info["start_time"] = datetime.now()
            subprocess.run(f"conda run -n {env} papermill --cwd '{run_folder}' '{script}' '{run_folder/script.name}'", shell=True, check=True)
            current_info["sucess"] = True
        except Exception as e:
            current_info["sucess"] = False
            print(f"Error during run number {row['run_num']}")
            display(e)  
        current_info["exec_end_time"] = datetime.now()
    else:
        current_info["sucess"] = "skipped"
    
    if (run_folder/script.name).exists() and not (run_folder/script.name).with_suffix(".html").exists():
        try:
            subprocess.run(f'jupyter nbconvert --to html {run_folder/script.name}', shell=True, check=True)
            current_info["html_sucess"] = True
        except Exception as e:
            print("Error while attempting to convert to notebook")
            display(e)
            current_info["html_sucess"] = False
        current_info["end_time"] = datetime.now()
    else:
        current_info["html_sucess"] = "skipped"
    if (run_folder / "__pycache__").exists():
        shutil.rmtree(run_folder / "__pycache__")
    if current_info["sucess"]==True and current_info["html_sucess"]==True:
        if dest_folder.exists():
            shutil.rmtree(dest_folder)
        shutil.move(run_folder, dest_folder)
    infos.append(current_info)
infos = pd.DataFrame(infos)
for col in ["start_time", "end_time"]:
    if not col in infos.columns:
        infos[col]=pd.NaT
for col in ["html_sucess"]:
    if not col in infos.columns:
        infos[col]=pd.NA
infos["duration"] = (infos["end_time"] - infos["start_time"]).dt.total_seconds()
infos=infos.drop(columns=["end_time"])
infos

In [ ]:
all = infos.merge(run_df[[col for col in run_df.columns if not col=="id"]], how="left", on=["run_num", "script_name"])
all["notebook_url"] = '<a href="'+ all.apply(lambda row: str((Path(row["run_folder"]) / row["script_name"]).with_suffix(".html").resolve()), axis=1) + '">notebook</a>'
all.insert(1, "notebook_url", all.pop("notebook_url").where((all["html_sucess"]==True) | (all["html_sucess"]=="skipped")))
all
# all.drop(columns="script_params")

In [ ]:
if "run_summary_folder" in params:
    all.to_csv(Path(ctx.evaluate(params["run_summary_folder"]))/"run_summary.tsv", sep="\t", index=False)
    